# Lab 1: Building Effective Prompts with the CRAFT Framework

In this lab, you'll learn to systematically construct high-quality prompts using the CRAFT framework. You'll measure prompt quality, build reusable templates, and visualize the difference between vague and well-structured prompts.

## Learning Objectives
- Apply the CRAFT framework systematically to any task
- Compare prompt quality before and after applying CRAFT
- Build reusable prompt templates for common tasks

**Duration:** 45 minutes | **Difficulty:** Beginner

## Part 1: Understanding Prompt Components

The CRAFT framework breaks every prompt into five essential components:

- **C**ontext: Background information the AI needs to understand the situation
- **R**ole: The expert persona the AI should adopt
- **A**ction: The specific task you want performed
- **F**ormat: How the output should be structured (bullets, table, essay, etc.)
- **T**one: The voice and style of the response

Let's build a Python toolkit to construct and evaluate CRAFT prompts.

In [ ]:
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple
import re
import textwrap
import matplotlib.pyplot as plt
import numpy as np


@dataclass
class CRAFTPrompt:
    """A structured prompt built using the CRAFT framework."""
    context: str
    role: str
    action: str
    format_spec: str
    tone: str

    def build(self) -> str:
        """Assemble components into a formatted prompt string."""
        sections = []
        if self.context:
            sections.append(f"Context: {self.context}")
        if self.role:
            sections.append(f"Role: Act as {self.role}.")
        if self.action:
            sections.append(f"Task: {self.action}")
        if self.format_spec:
            sections.append(f"Format: {self.format_spec}")
        if self.tone:
            sections.append(f"Tone: {self.tone}")
        return "\n\n".join(sections)

    def summary(self) -> str:
        """Return a one-line summary showing which fields are filled."""
        filled = []
        for label, val in [("C", self.context), ("R", self.role),
                           ("A", self.action), ("F", self.format_spec),
                           ("T", self.tone)]:
            status = "YES" if val.strip() else "---"
            filled.append(f"{label}:{status}")
        return "  |  ".join(filled)


def score_prompt(prompt_text: str) -> Dict[str, int]:
    """Score a prompt from 1-5 on five quality dimensions.

    Dimensions:
        clarity       - Is the request unambiguous?
        specificity   - Does it include concrete details?
        context       - Does it provide background information?
        format        - Does it specify desired output structure?
        actionability - Is there a clear, measurable action?

    Scoring uses keyword and pattern analysis as a lightweight heuristic.
    """
    text = prompt_text.lower()
    words = text.split()
    word_count = len(words)

    # --- Clarity ---
    clarity = 1
    if word_count >= 10:
        clarity += 1
    if any(w in text for w in ["specifically", "exactly", "precisely", "must", "should"]):
        clarity += 1
    if not any(w in text for w in ["something", "stuff", "things", "whatever", "etc"]):
        clarity += 1
    if word_count >= 25:
        clarity += 1
    clarity = min(clarity, 5)

    # --- Specificity ---
    specificity = 1
    if re.search(r'\d+', text):
        specificity += 1
    if any(w in text for w in ["example", "such as", "including", "e.g.", "for instance"]):
        specificity += 1
    specific_nouns = ["industry", "sector", "company", "product", "metric",
                      "audience", "stakeholder", "customer", "market", "region"]
    if sum(1 for w in specific_nouns if w in text) >= 2:
        specificity += 1
    if word_count >= 30:
        specificity += 1
    specificity = min(specificity, 5)

    # --- Context ---
    context_score = 1
    context_signals = ["background", "context", "situation", "scenario",
                       "currently", "our company", "the team", "we are",
                       "given that", "assuming", "based on"]
    matches = sum(1 for s in context_signals if s in text)
    context_score += min(matches, 2)
    if word_count >= 20:
        context_score += 1
    if re.search(r'(role|act as|you are|persona)', text):
        context_score += 1
    context_score = min(context_score, 5)

    # --- Format ---
    format_score = 1
    format_signals = ["bullet", "numbered", "table", "list", "heading",
                      "paragraph", "json", "csv", "markdown", "format",
                      "structure", "outline", "section", "step-by-step"]
    fmt_matches = sum(1 for s in format_signals if s in text)
    format_score += min(fmt_matches, 2)
    if re.search(r'\d+\s*(words|sentences|paragraphs|points|items|bullets)', text):
        format_score += 1
    if any(w in text for w in ["include", "exclude", "begin with", "end with"]):
        format_score += 1
    format_score = min(format_score, 5)

    # --- Actionability ---
    actionability = 1
    action_verbs = ["write", "create", "generate", "analyze", "compare",
                    "summarize", "evaluate", "list", "design", "draft",
                    "build", "develop", "produce", "identify", "recommend"]
    verb_hits = sum(1 for v in action_verbs if v in text)
    actionability += min(verb_hits, 2)
    if re.search(r'(for|targeting|aimed at)\s+\w+', text):
        actionability += 1
    if word_count >= 15:
        actionability += 1
    actionability = min(actionability, 5)

    scores = {
        "clarity": clarity,
        "specificity": specificity,
        "context": context_score,
        "format": format_score,
        "actionability": actionability,
    }

    total = sum(scores.values())
    max_total = 25

    print(f"{'Dimension':<16} {'Score':>5}")
    print("-" * 23)
    for dim, val in scores.items():
        bar = '#' * val + '.' * (5 - val)
        print(f"{dim:<16} [{bar}] {val}/5")
    print("-" * 23)
    print(f"{'TOTAL':<16} {total}/{max_total}")
    print()

    return scores


# Quick sanity check
print("=== Scoring a vague prompt ===")
score_prompt("Write about AI")

print("=== Scoring a detailed CRAFT prompt ===")
score_prompt(
    "Context: Our healthcare startup is preparing a board presentation. "
    "Role: Act as a healthcare strategy consultant with 15 years experience. "
    "Task: Write a 500-word executive summary analyzing AI adoption trends "
    "in the healthcare industry, including 3 specific examples of successful "
    "implementations. Format: Use bullet points for key findings followed by "
    "a numbered list of recommendations. Tone: Professional and data-driven."
)

## Part 2: Before & After Analysis

The real power of CRAFT becomes clear when you compare a raw, off-the-cuff prompt with its structured counterpart. Below we take three common but vague prompts, rebuild each one with CRAFT, and measure the difference.

In [ ]:
# ---------- Before / After pairs ----------

before_after: List[Tuple[str, CRAFTPrompt]] = [
    # Pair 1 -- "Write about AI"
    (
        "Write about AI",
        CRAFTPrompt(
            context="Our hospital network is evaluating AI tools to reduce "
                    "diagnostic errors. The audience is the clinical leadership team.",
            role="a healthcare technology analyst with expertise in clinical AI",
            action="Write a 600-word briefing on how AI-assisted diagnostics are "
                   "reducing misdiagnosis rates, including 3 specific case studies "
                   "from peer-reviewed research.",
            format_spec="Start with a 2-sentence executive summary, then use "
                        "bullet points for each case study, and end with a "
                        "numbered list of 3 recommendations.",
            tone="Professional, evidence-based, and persuasive"
        )
    ),
    # Pair 2 -- "Summarize this"
    (
        "Summarize this",
        CRAFTPrompt(
            context="The attached 40-page quarterly earnings report covers "
                    "revenue, operating costs, and forward guidance for Q3 2025.",
            role="a senior financial analyst preparing materials for the CFO",
            action="Summarize the report into an executive briefing that "
                   "highlights the 5 most critical data points, identifies "
                   "2 risks, and notes any upward or downward trends.",
            format_spec="Use a structured format: one heading per section, "
                        "bullet points for data, and a comparison table for "
                        "quarter-over-quarter metrics. Keep it under 300 words.",
            tone="Concise, analytical, and neutral"
        )
    ),
    # Pair 3 -- "Help with email"
    (
        "Help with email",
        CRAFTPrompt(
            context="We delivered a software demo to a prospective enterprise "
                    "client last Tuesday. They asked about SSO integration "
                    "and data residency in the EU. We need to follow up "
                    "within 48 hours.",
            role="an enterprise account executive at a B2B SaaS company",
            action="Draft a follow-up email that thanks the client for their "
                   "time, directly addresses their SSO and data-residency "
                   "questions, and proposes a 30-minute technical deep-dive "
                   "call next week.",
            format_spec="Subject line + body. Body should have a greeting, "
                        "3 short paragraphs, and a clear call-to-action with "
                        "2 proposed meeting times.",
            tone="Warm, confident, and professional"
        )
    ),
]

# ---------- Score and display comparison ----------

all_before_scores: List[Dict[str, int]] = []
all_after_scores: List[Dict[str, int]] = []

for idx, (before_text, after_craft) in enumerate(before_after, start=1):
    after_text = after_craft.build()

    print("=" * 60)
    print(f"  PAIR {idx}")
    print("=" * 60)

    print(f"\n--- BEFORE ---")
    print(f'  "{before_text}"\n')
    b_scores = score_prompt(before_text)

    print(f"--- AFTER (CRAFT) ---")
    print(textwrap.indent(after_text, "  ") + "\n")
    a_scores = score_prompt(after_text)

    improvement = sum(a_scores.values()) - sum(b_scores.values())
    print(f"  >>> Improvement: +{improvement} points\n")

    all_before_scores.append(b_scores)
    all_after_scores.append(a_scores)

# ---------- Summary table ----------
print("\n" + "=" * 60)
print("  SUMMARY TABLE")
print("=" * 60)
header = f"{'Pair':<6} {'Before':>8} {'After':>8} {'Delta':>8}"
print(header)
print("-" * len(header))
for i in range(len(before_after)):
    b_total = sum(all_before_scores[i].values())
    a_total = sum(all_after_scores[i].values())
    print(f"{i+1:<6} {b_total:>7}/25 {a_total:>7}/25 {'+' + str(a_total - b_total):>7}")

## Exercise 1: Score Your Own Prompts

Think of a task you perform regularly at work (writing reports, drafting emails, analyzing data, creating presentations). Build a CRAFT prompt for it and score the result.

**Goal:** Achieve a total score of at least 18/25.

In [ ]:
# YOUR CODE HERE
# Create a CRAFTPrompt for a task relevant to your work
# Then score it using score_prompt()

my_prompt = CRAFTPrompt(
    context="",      # Fill in: What background does the AI need?
    role="",         # Fill in: What expert should the AI be?
    action="",       # Fill in: What specific task?
    format_spec="",  # Fill in: How should output be structured?
    tone=""          # Fill in: What voice/style?
)

print("CRAFT Component Check:")
print(my_prompt.summary())
print()

print("Your CRAFT Prompt:")
print(my_prompt.build())
print()

print("Score:")
score_prompt(my_prompt.build())

## Part 3: Prompt Templates

Instead of crafting every prompt from scratch, experienced prompt engineers maintain a **template library** of reusable CRAFT prompts with placeholders. This section introduces a `PromptLibrary` class that lets you store, retrieve, list, and fill templates with specific values.

In [ ]:
from collections import OrderedDict


class PromptLibrary:
    """A reusable library of CRAFT prompt templates."""

    def __init__(self):
        self._templates: OrderedDict[str, CRAFTPrompt] = OrderedDict()

    def add_template(self, name: str, prompt: CRAFTPrompt) -> None:
        """Register a new template."""
        self._templates[name] = prompt
        print(f"  [+] Template '{name}' added.")

    def get_template(self, name: str) -> CRAFTPrompt:
        """Retrieve a template by name."""
        if name not in self._templates:
            raise KeyError(f"Template '{name}' not found. "
                           f"Available: {list(self._templates.keys())}")
        return self._templates[name]

    def list_templates(self) -> None:
        """Display all registered templates."""
        print(f"{'#':<4} {'Template Name':<25} {'Placeholders'}")
        print("-" * 65)
        for i, (name, tmpl) in enumerate(self._templates.items(), 1):
            combined = tmpl.build()
            placeholders = sorted(set(re.findall(r'\{(\w+)\}', combined)))
            print(f"{i:<4} {name:<25} {', '.join(placeholders) or '(none)'}")

    def fill_template(self, name: str, **kwargs: str) -> str:
        """Fill placeholders in a template and return the final prompt."""
        tmpl = self.get_template(name)
        raw = tmpl.build()

        # Find unfilled placeholders
        expected = set(re.findall(r'\{(\w+)\}', raw))
        missing = expected - set(kwargs.keys())
        if missing:
            print(f"  Warning: unfilled placeholders: {missing}")

        # Safe formatting that ignores unknown keys
        for key, value in kwargs.items():
            raw = raw.replace("{" + key + "}", value)
        return raw


# ---------- Pre-populate the library ----------

library = PromptLibrary()

library.add_template("email_writer", CRAFTPrompt(
    context="We are a {company_type} company. The recipient is {recipient}.",
    role="a professional communications specialist",
    action="Draft a {email_type} email about {topic} that includes a clear "
           "call-to-action.",
    format_spec="Subject line + 3 concise paragraphs. Keep under 200 words.",
    tone="{tone}"
))

library.add_template("code_reviewer", CRAFTPrompt(
    context="The code is written in {language} for a {project_type} project. "
           "The team follows {standard} coding standards.",
    role="a senior software engineer and code reviewer",
    action="Review the following code for bugs, performance issues, and "
           "readability. Identify at least 3 specific improvements.",
    format_spec="Use a numbered list. For each issue: state the problem, "
                "show the offending line, and suggest a fix with a code snippet.",
    tone="Constructive and educational"
))

library.add_template("meeting_summarizer", CRAFTPrompt(
    context="The meeting was a {meeting_type} attended by {attendees}. "
           "Duration: {duration}.",
    role="an executive assistant skilled at distilling meeting notes",
    action="Summarize the meeting into key decisions, action items with "
           "owners, and open questions. Flag any items that are blocked.",
    format_spec="Use headings: Decisions, Action Items (table with Owner and "
                "Due Date columns), Open Questions. Keep under 250 words.",
    tone="Neutral and precise"
))

library.add_template("data_analyst", CRAFTPrompt(
    context="The dataset contains {dataset_description}. The stakeholder "
           "is {audience}.",
    role="a senior data analyst at a {industry} company",
    action="Analyze the data to identify the top {num_insights} insights, "
           "highlight any anomalies, and recommend next steps.",
    format_spec="Executive summary (3 sentences), then bullet points for each "
                "insight, then a table of anomalies with severity ratings.",
    tone="Data-driven and actionable"
))

# ---------- Show the library ----------
print("\n")
library.list_templates()

# ---------- Demo: fill a template ----------
print("\n" + "=" * 60)
print("  DEMO: Filling the 'email_writer' template")
print("=" * 60 + "\n")

filled = library.fill_template(
    "email_writer",
    company_type="B2B SaaS",
    recipient="the VP of Engineering at a Fortune 500 client",
    email_type="follow-up",
    topic="the security audit results from last week",
    tone="Confident, professional, and reassuring"
)
print(filled)
print("\n--- Score ---")
score_prompt(filled)

## Exercise 2: Build Your Template Library

Add **two new templates** to the library for tasks you perform regularly. Use `{placeholders}` for values that change between uses, then demonstrate each template by filling it in.

**Ideas:** social media post, project status update, customer FAQ answer, training outline, bug report, interview question set.

In [ ]:
# YOUR CODE HERE
# Add 2 new templates to the library for tasks you do regularly
# Then use fill_template() to generate a complete prompt from each one

library.add_template(
    "your_template_1",
    CRAFTPrompt(
        context="",      # Include {placeholders} for variable parts
        role="",
        action="",
        format_spec="",
        tone=""
    )
)

library.add_template(
    "your_template_2",
    CRAFTPrompt(
        context="",
        role="",
        action="",
        format_spec="",
        tone=""
    )
)

# Fill and display template 1
print("=== Template 1 ===")
result1 = library.fill_template("your_template_1")  # add your kwargs
print(result1)
print()

# Fill and display template 2
print("=== Template 2 ===")
result2 = library.fill_template("your_template_2")  # add your kwargs
print(result2)
print()

# Show the updated library
print("\n=== Full Library ===")
library.list_templates()

## Part 4: Visualization - Prompt Quality Comparison

A radar chart (spider chart) makes it easy to see which quality dimensions are strong or weak across different prompt styles. Below we compare three prompts representing increasing levels of structure:

1. **Vague** - A bare-minimum, one-line request
2. **Partially Structured** - Some detail but missing key CRAFT components
3. **Full CRAFT** - All five components filled in

In [ ]:
# ---------- Three comparison prompts ----------

vague_prompt = "Make a marketing plan"

partial_prompt = (
    "Write a marketing plan for our new mobile app targeting millennials. "
    "Include social media strategy and budget estimates."
)

craft_prompt = CRAFTPrompt(
    context="Our fintech startup is launching a budgeting app aimed at "
            "millennials (ages 25-40) in the US market. We have a quarterly "
            "marketing budget of $50,000 and currently 2,000 beta users.",
    role="a digital marketing strategist with 10 years of experience in "
         "fintech product launches",
    action="Create a 90-day go-to-market plan that identifies the top 3 "
           "customer acquisition channels, proposes a content calendar, and "
           "estimates cost-per-acquisition for each channel.",
    format_spec="Use the following structure: Executive Summary (5 sentences), "
                "Channel Strategy (table with columns: Channel, Audience Fit, "
                "Monthly Budget, Est. CPA), Content Calendar (bullet points "
                "by week), and KPIs (numbered list of 5 metrics to track).",
    tone="Strategic, data-informed, and action-oriented"
).build()

# ---------- Score each ----------
print("Scoring: Vague prompt")
s_vague = score_prompt(vague_prompt)
print("Scoring: Partial prompt")
s_partial = score_prompt(partial_prompt)
print("Scoring: Full CRAFT prompt")
s_craft = score_prompt(craft_prompt)

# ---------- Radar chart ----------
dimensions = list(s_vague.keys())
n_dims = len(dimensions)

angles = np.linspace(0, 2 * np.pi, n_dims, endpoint=False).tolist()
angles += angles[:1]  # close the polygon

def values_for(scores: Dict[str, int]) -> List[int]:
    vals = [scores[d] for d in dimensions]
    return vals + vals[:1]

fig, ax = plt.subplots(figsize=(7, 7), subplot_kw=dict(polar=True))

ax.plot(angles, values_for(s_vague), 'o-', linewidth=2,
        label=f'Vague ({sum(s_vague.values())}/25)', color='#e74c3c')
ax.fill(angles, values_for(s_vague), alpha=0.10, color='#e74c3c')

ax.plot(angles, values_for(s_partial), 's-', linewidth=2,
        label=f'Partial ({sum(s_partial.values())}/25)', color='#f39c12')
ax.fill(angles, values_for(s_partial), alpha=0.10, color='#f39c12')

ax.plot(angles, values_for(s_craft), 'D-', linewidth=2,
        label=f'Full CRAFT ({sum(s_craft.values())}/25)', color='#27ae60')
ax.fill(angles, values_for(s_craft), alpha=0.15, color='#27ae60')

ax.set_xticks(angles[:-1])
ax.set_xticklabels([d.capitalize() for d in dimensions], fontsize=12)
ax.set_yticks([1, 2, 3, 4, 5])
ax.set_yticklabels(['1', '2', '3', '4', '5'], fontsize=9, color='grey')
ax.set_ylim(0, 5)
ax.set_title('Prompt Quality Comparison', fontsize=15, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=11)

plt.tight_layout()
plt.show()

## Challenge: The Prompt Makeover

Below are three intentionally terrible prompts. Your mission:

1. Score each one as-is
2. Transform each into a full CRAFT prompt
3. Score the transformed versions
4. Visualize all six scores (3 before + 3 after) in a grouped bar chart

**Target:** Every transformed prompt should score at least 20/25.

In [ ]:
# YOUR CODE HERE - Challenge Exercise
# Take these 3 terrible prompts and transform them using CRAFT
# Score each before and after, and visualize the improvement

terrible_prompts = [
    "Make a presentation about sales",
    "Fix my code",
    "Write something for social media"
]

# Step 1: Score the terrible prompts
before_scores = []
for prompt in terrible_prompts:
    print(f'Scoring: "{prompt}"')
    before_scores.append(score_prompt(prompt))

# Step 2: Create CRAFT versions
# craft_version_1 = CRAFTPrompt(context="...", role="...", action="...",
#                                format_spec="...", tone="...")
# craft_version_2 = CRAFTPrompt(...)
# craft_version_3 = CRAFTPrompt(...)

# Step 3: Score the CRAFT versions
# after_scores = []
# for craft_ver in [craft_version_1, craft_version_2, craft_version_3]:
#     after_scores.append(score_prompt(craft_ver.build()))

# Step 4: Visualize with a grouped bar chart
# Hint: use plt.bar() with offset x positions for before/after groups
# Labels for each prompt pair on the x-axis
# Y-axis = total score out of 25